## 해당 파일은 자연어처리를 사용해 텍스트를 정량적으로 처리하는 부분이다.
1. 가상의 웹 쇼핑몰 페이지를 가정, 물품에 달린 리뷰댓글을 토대로 감정분석을 통해 물품이 사용자에게 어떤 반응을 얻고있는지 통계를 낸다.
2. 제품 페이지 내 신규 등록된 리뷰 댓글의 감정 분석을 처리해 기존의 통계 누적 평균을 도출하는 식으로 표현 될 예정이다. 따라서, 불필요하게 연산을 더 많이 하거나 무거운 NLP 의 처리가 다회 이루어질 필요가 없다.
3. 알고리즘의 형태는 감정 분석 : Sentiment Analysis 로 사전 학습된 한국어 자연어 처리 모델을 사용한다. 
4. 감성분석의 결과는 역접에 의한 통계적 분석이 오류가 되는 경우가 많다, 따라서 역접을 기준으로 하여 문장의 앞뒤를 분리 및 분석을 시도한다.
5. 분절된 문장의 긍/부정 평균수치를 계산하여 해당 리뷰의 감성적 평점을 산출한다.
6. 제품에 대한 전체적인 사용자의 종합적 점수 판단을 위해 n 개의 리뷰가 가질 수 있는 최대점 5.0 을 기준으로 평균을 내어 해당 제품의 소비자 반응 긍정도와 감정 평균 점수의 분포도를 확인 할 수 있도록 한다.
7. 오류를 최소화 하기위해 리뷰의 길이 평균을 적용시켜, 일정 길이 미만의 후기들의 감성 점수는 차등 적용키로 한다.
8. 추후 제품의 종류, 즉 카테고리 기능이 포함된다면 판매회수와 누적 리뷰 횟수에 따라 만족도를 차등 가중시켜 판매 인기도가 높은 순서의 정렬이 가능하게 시도한다.
9. 사용자의 카테고리별 리뷰 트렌트에 맞추어 GNN 과 같은 추천 알고리즘은 아니어도 간단한 선형 모델의 결합으로 사용자에게 높은 리뷰점수를 받은 제품을 사전에 소개할 수 있다.

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/MyDrive/Colab_Projects/22_캡스톤디자인/Module')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 최초 실행시 구동 요망

In [2]:
#my_path = '/content/drive/MyDrive/Colab_Projects/22_캡스톤디자인/Module'
#!pip install --target=$my_path transformers
#!pip install import_ipynb

## 모델링 개요
1. 하이퍼 클로바 등과 같이 최근 한국어를 다룬 AI 트렌트에 맞추어 SKT 사의 KoGPT2 모델을 활용할 예정에 있다.
2. Pretrained weight 를 활용해 기존 40GB 에 달하는 사전 데이터에서 10% 달하는 4GB 정도의 텍스트 리뷰데이터를 크롤링하여 Fine Tuned을 진행해본다
3. 각 리뷰의 평균 점수를 총 인원의 계로 나타낼땐 역시 상기된 바와 동일하게 차등 가중치 방식으로 빈도 분포에 따른 다항식을 적용하고자 한다. ( 이는 리뷰 텍스트가 짧을 수록 감성 정확도가 낮아지는 것에 이유를 둔다 )

### Google 의 Electra 변형
1. Replaced Token Detection 즉 생선된 토큰으로부터 legit or fake 여부를 판단하며 모든 input token에 대한 학습이 가능하여, KoBert 등과의 비교에 더 좋은 성능을 내었다.
2. 해당 모델의 WordPiece 러닝은 형태소 분석 없이 캐릭터 토큰사의의 관계를 파악하기 용이하고, 다양한 언어에 적용이 쉽기때문에 본 프로젝트에 사용되었다.

In [3]:
%cd /content/drive/MyDrive/Colab_Projects/22_캡스톤디자인
!ls

/content/drive/MyDrive/Colab_Projects/22_캡스톤디자인
function_modules.ipynb	Module	nlp_proccessing_unit.ipynb


In [4]:
import torch
import import_ipynb
from function_modules import *

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

importing Jupyter notebook from function_modules.ipynb


In [5]:
tokenizer = AutoTokenizer.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

def setiment_scoring(sentence):
  """ sentence 내의 데이터가 0 일 경우, 0을 리턴해 모델의 실행을 피한다.
  현재 텍스트 데이터를 분절하고 빈 곳은 0으로 채워져 있기 때문이다. """
  if not sentence:
    return 0
  
  # 모듈의 sentence 값에 따른 prediction 은 딕셔너리 형태로 리턴이 된다.
  temp_scores = sentiment_classifier(sentence)
  temp_scores = list(temp_scores[0].values())

  """딕셔너리의 value 값 [0]은 긍/부정, [1]은 확실성을 나타낸다.
  따라서 긍정일 경우 그대로, 부정일 경우 음수로 변환하여 확실성 평균을 낸다."""
  if temp_scores[0] == '1':
    return temp_scores[1]
  else:
    return -temp_scores[1]

### 업데이트 사항
1. SKT의 KoGPT2 텍스트 본문 생성하는 것에 주안점이 맞추어져 있어 감성분석 사전학습된 모델인 SKT의 Electra 를 사용하는 것으로 대체되었다.
2. 러닝을 시킬 자원이 용이치 않아 구글의 Electra Base-v3 모델을 한국어에 사전학습시킨 HuggingFace에 등재된 KoElectra Base-v3 모델 모듈을 사용해보았다.
3. 분절의 기준은 리뷰가 여러 문장으로 이루어진 문단이라는 점에서 마침표를 기준으로 변경되었다.
4. 리스트 구조보다 DataFrame과 Series의 활용으로 공간, 시간 복잡도를 더욱 효율적으로 사용하게 조정하였다.
5. 길이에 따른 평균수치의 차등적 적용을 위해 최소길이, 최대길이 임계값을 지정하였다, 이는 추후 사용자가 원하는 임계값에 따라 변경될 수 있도록 하였다. { default : 25 < length < 100 }
6. 리뷰 데이터 셋이 커질수록 for 와 같은 반복문의 유무가 시간적 문제가 되기 때문에 series 함수와 apply를 활용해 최대한 효율적으로 만듦, 실시간 피드백을 가능케 하기 위함.
7. 마침표에 따라 문장의 분절을 하기위함이지만, 문장의 끝에 마침표가 의무적으로 들어가는 것을 전처리함.

## 정량화 알고리즘 개요

### 텍스트 데이터 분절 처리

In [6]:
review_list = [
               '핵불닭은 맵긴한데 중독성있어서 좋아요',
               '맛있는지는 잘 모르겠네요',
               '너무 짠거 아니에요? 소스 때문에 살찌겠어요',
               '닭가슴살 퍽퍽해서 맛대가리 없어요',
               '다음에는 안삽니다 맛대가리 없네요',
               '이건 먹고싶었는데 항상 품절이여서 못먹었다가 이번에 기대하면서 시켰는데 진짜로 양념치킨맛이 나서 너무 감사했던 닭가슴살이였음..기대이상 인 맛이여서 행복했다 ㅠ',
               '촉촉하고 양념도 맛있어서 잘 들어가네요! 잘먹겠습니다',
               '다 좋은데 닭가슴살이 너무 짜요 ㅠㅠ',
               '양념 그대로 밥이랑 먹으면 진짜 그냥 치밥! 그냥 먹어도 부드럽고 밥이랑ㅈ볶아 먹어도 맛있어요.',
               '맛있어요 재주문 할거에요.',
               '젓가락으로 찢어서 소스에 비벼먹으면 맛있네요!! 개인적으로 핵불닭이 제일 맛있어요.',
               '한끼 통살 덕분에 식단할 맛이 나요. 진짜 맛있어요.',
               '닭가슴살이 이렇게 맛있을수가 있구나 싶을정도로 전맛탱이에용 내돈내산임! 근데 데리야끼맛 하나가 터져서 왔는지 양념이 새서 엄청 끈적끈적 햏네요 ㅠㅠ교환이나 반품 귀찮아서 그냥 먹었는데 포장 좀만 더 신경써주시면 좋을것같아요~ 재주문할개요^^',
               '맛있어요 또 시킬거에요 존맛 하나도 버릴게 없는맛',
               '딱딱할거라고 생각했는데 훨씬부드럽고 맛있어요 소스도 맛있어요',
               '닭이 너무 부드럽고 맛있어요 짜지않은 제품울 원했는데 타사제품보다 덜 짜요 소스때문에 어쩔 수 없을거 같아요 샐러드나 현미밥에 먹기 좋은 간입니다 다만 한봉지 분량 넘나 작아서 한끼에 두개씩 뜯어먹어야하구요 잘려있지 않는 제품이라 밖에 가지고 나갈때는 도시락에 잘라넣어서 다녀요 소스가 있기때문에 봉지째 들고나가면 불편함이 없지않아 있습니다',
               '너무 매워요 맵찔이는 좀 힘들듯',
               '파스타맛같아요 양념포함이라 너무 맛있게 먹고있어요! 전 매콤토마토랑 흑마늘이 젤 좋더라구요 ㅎㅎ 까르보맛하고 갈릭은 쏘쏘였구 흑마늘하고 매콤 토마토 계속 재주문중이요 ㅎㅎ',
               '저는다이어트 할때마다 여기서먹어요 뭐닭가슴살이 살이빠지니마니해도 ㅋㅋ저는 먹는데 진심이고 많이먹는타입이고 식욕을 못참은 여자로써 여기닭가슴살이랑 볶음밥은 그냥 일반음식같아요 넘넘맛있어요요요 외식이랑 배달이 확줄엇어요 저는 별다섯개입니다',
               '별로 맛있진않음',
               '1팩 추가 증정 된다고 해서 시킨 건데 장바구니에 담아놓은 동안 이벤트 기간이 지났나봐요.추가 증정품은 없네요.단가가 비싼 편이고, 상품사진처럼 깔끔하게 먹기에는 소스 때문에 좀 어려워요.다양한 맛을 즐길 수 있는 점은 좋아요.',
               '누락되어서 매번 문의글올리고. 택배 두번받고. 정말 불만족스러워요.',
               '핵불닭은 맵긴한데 중독성있어서 좋아요',
               '맛있는지는 잘 모르겠네요',
               '너무 짠거 아니에요? 소스 때문에 살찌겠어요',
               '닭가슴살 퍽퍽해서 맛대가리 없어요',
               '다음에는 안삽니다 맛대가리 없네요',
               '이건 먹고싶었는데 항상 품절이여서 못먹었다가 이번에 기대하면서 시켰는데 진짜로 양념치킨맛이 나서 너무 감사했던 닭가슴살이였음..기대이상 인 맛이여서 행복했다 ㅠ',
               '촉촉하고 양념도 맛있어서 잘 들어가네요! 잘먹겠습니다',
               '다 좋은데 닭가슴살이 너무 짜요 ㅠㅠ',
               '양념 그대로 밥이랑 먹으면 진짜 그냥 치밥! 그냥 먹어도 부드럽고 밥이랑ㅈ볶아 먹어도 맛있어요.',
               '맛있어요 재주문 할거에요.',
               '젓가락으로 찢어서 소스에 비벼먹으면 맛있네요!! 개인적으로 핵불닭이 제일 맛있어요.',
               '한끼 통살 덕분에 식단할 맛이 나요. 진짜 맛있어요.',
               '닭가슴살이 이렇게 맛있을수가 있구나 싶을정도로 전맛탱이에용 내돈내산임! 근데 데리야끼맛 하나가 터져서 왔는지 양념이 새서 엄청 끈적끈적 햏네요 ㅠㅠ교환이나 반품 귀찮아서 그냥 먹었는데 포장 좀만 더 신경써주시면 좋을것같아요~ 재주문할개요^^',
               '맛있어요 또 시킬거에요 존맛 하나도 버릴게 없는맛',
               '딱딱할거라고 생각했는데 훨씬부드럽고 맛있어요 소스도 맛있어요',
               '닭이 너무 부드럽고 맛있어요 짜지않은 제품울 원했는데 타사제품보다 덜 짜요 소스때문에 어쩔 수 없을거 같아요 샐러드나 현미밥에 먹기 좋은 간입니다 다만 한봉지 분량 넘나 작아서 한끼에 두개씩 뜯어먹어야하구요 잘려있지 않는 제품이라 밖에 가지고 나갈때는 도시락에 잘라넣어서 다녀요 소스가 있기때문에 봉지째 들고나가면 불편함이 없지않아 있습니다',
               '너무 매워요 맵찔이는 좀 힘들듯',
               '파스타맛같아요 양념포함이라 너무 맛있게 먹고있어요! 전 매콤토마토랑 흑마늘이 젤 좋더라구요 ㅎㅎ 까르보맛하고 갈릭은 쏘쏘였구 흑마늘하고 매콤 토마토 계속 재주문중이요 ㅎㅎ',
               '저는다이어트 할때마다 여기서먹어요 뭐닭가슴살이 살이빠지니마니해도 ㅋㅋ저는 먹는데 진심이고 많이먹는타입이고 식욕을 못참은 여자로써 여기닭가슴살이랑 볶음밥은 그냥 일반음식같아요 넘넘맛있어요요요 외식이랑 배달이 확줄엇어요 저는 별다섯개입니다',
               '별로 맛있진않음',
               '1팩 추가 증정 된다고 해서 시킨 건데 장바구니에 담아놓은 동안 이벤트 기간이 지났나봐요.추가 증정품은 없네요.단가가 비싼 편이고, 상품사진처럼 깔끔하게 먹기에는 소스 때문에 좀 어려워요.다양한 맛을 즐길 수 있는 점은 좋아요.',
               '누락되어서 매번 문의글올리고. 택배 두번받고. 정말 불만족스러워요.',
               '핵불닭은 맵긴한데 중독성있어서 좋아요',
               '맛있는지는 잘 모르겠네요',
               '너무 짠거 아니에요? 소스 때문에 살찌겠어요',
               '닭가슴살 퍽퍽해서 맛대가리 없어요',
               '다음에는 안삽니다 맛대가리 없네요',
               '이건 먹고싶었는데 항상 품절이여서 못먹었다가 이번에 기대하면서 시켰는데 진짜로 양념치킨맛이 나서 너무 감사했던 닭가슴살이였음..기대이상 인 맛이여서 행복했다 ㅠ',
               '촉촉하고 양념도 맛있어서 잘 들어가네요! 잘먹겠습니다',
               '다 좋은데 닭가슴살이 너무 짜요 ㅠㅠ',
               '양념 그대로 밥이랑 먹으면 진짜 그냥 치밥! 그냥 먹어도 부드럽고 밥이랑ㅈ볶아 먹어도 맛있어요.',
               '맛있어요 재주문 할거에요.',
               '젓가락으로 찢어서 소스에 비벼먹으면 맛있네요!! 개인적으로 핵불닭이 제일 맛있어요.',
               '한끼 통살 덕분에 식단할 맛이 나요. 진짜 맛있어요.',
               '닭가슴살이 이렇게 맛있을수가 있구나 싶을정도로 전맛탱이에용 내돈내산임! 근데 데리야끼맛 하나가 터져서 왔는지 양념이 새서 엄청 끈적끈적 햏네요 ㅠㅠ교환이나 반품 귀찮아서 그냥 먹었는데 포장 좀만 더 신경써주시면 좋을것같아요~ 재주문할개요^^',
               '맛있어요 또 시킬거에요 존맛 하나도 버릴게 없는맛',
               '딱딱할거라고 생각했는데 훨씬부드럽고 맛있어요 소스도 맛있어요',
               '닭이 너무 부드럽고 맛있어요 짜지않은 제품울 원했는데 타사제품보다 덜 짜요 소스때문에 어쩔 수 없을거 같아요 샐러드나 현미밥에 먹기 좋은 간입니다 다만 한봉지 분량 넘나 작아서 한끼에 두개씩 뜯어먹어야하구요 잘려있지 않는 제품이라 밖에 가지고 나갈때는 도시락에 잘라넣어서 다녀요 소스가 있기때문에 봉지째 들고나가면 불편함이 없지않아 있습니다',
               '너무 매워요 맵찔이는 좀 힘들듯',
               '파스타맛같아요 양념포함이라 너무 맛있게 먹고있어요! 전 매콤토마토랑 흑마늘이 젤 좋더라구요 ㅎㅎ 까르보맛하고 갈릭은 쏘쏘였구 흑마늘하고 매콤 토마토 계속 재주문중이요 ㅎㅎ',
               '저는다이어트 할때마다 여기서먹어요 뭐닭가슴살이 살이빠지니마니해도 ㅋㅋ저는 먹는데 진심이고 많이먹는타입이고 식욕을 못참은 여자로써 여기닭가슴살이랑 볶음밥은 그냥 일반음식같아요 넘넘맛있어요요요 외식이랑 배달이 확줄엇어요 저는 별다섯개입니다',
               '별로 맛있진않음',
               '1팩 추가 증정 된다고 해서 시킨 건데 장바구니에 담아놓은 동안 이벤트 기간이 지났나봐요.추가 증정품은 없네요.단가가 비싼 편이고, 상품사진처럼 깔끔하게 먹기에는 소스 때문에 좀 어려워요.다양한 맛을 즐길 수 있는 점은 좋아요.',
               '누락되어서 매번 문의글올리고. 택배 두번받고. 정말 불만족스러워요.',
               ]

review, length_series = preprocessing(review_list)

### 모델의 점수 처리




In [7]:
def scoring(dataframe, series):
  list_of_col = list(dataframe.columns)
  list_of_col

  for col_no in list_of_col:
    dataframe[col_no] = dataframe[col_no].apply(setiment_scoring)
  
  avg_list = dataframe.apply(mean, axis = 1)
  weight_list = series.apply(threshold_computator)

  sentiment_score = (avg_list * weight_list).mean()
  sentiment_score = sentiment_total(sentiment_score)
  total_reviewers = len(avg_list)

  return sentiment_score, total_reviewers

In [8]:
sentiment, total_review  = scoring(review, length_series)

### 모델의 리뷰 데이터 총 진행 후 결과

In [9]:
print("총 ", total_review, "명의 고객님들이 ", sentiment, " 으로 반응을 하셨습니다 📑👍")

총  66 명의 고객님들이  대체로 긍정적  으로 반응을 하셨습니다 📑👍


In [10]:
del[[review, length_series]]